In [1]:
import os
import requests
from requests_html import HTML
import json
import time
import numpy as np
import pandas as pd
#os.chdir('D:\\')

In [2]:
"""import requests
from requests_html import HTML
import json
import time

base_url = 'https://stackoverflow.com/questions/tagged/'
tag = 'python'
query_filter = 'Votes'
url = f"{base_url}{tag}?tab={query_filter}"
url"""

'import requests\nfrom requests_html import HTML\nimport json\nimport time\n\nbase_url = \'https://stackoverflow.com/questions/tagged/\'\ntag = \'python\'\nquery_filter = \'Votes\'\nurl = f"{base_url}{tag}?tab={query_filter}"\nurl'

In [3]:
import requests
from requests_html import HTML
import json

base_url = 'https://stackoverflow.com/questions'
query_filter = 'Votes'
url = f"{base_url}?tab={query_filter}"
url

'https://stackoverflow.com/questions?tab=Votes'

In [4]:
r = requests.get(url)
html_str = r.text

In [5]:
html = HTML(html=html_str)

In [6]:
question_summaries = html.find(".s-post-summary")
print(question_summaries)

[<Element 'div' id='question-summary-11227809' class=('s-post-summary', 'js-post-summary') data-post-id='11227809' data-post-type-id='1'>, <Element 'div' id='question-summary-927358' class=('s-post-summary', 'js-post-summary') data-post-id='927358' data-post-type-id='1'>, <Element 'div' id='question-summary-2003505' class=('s-post-summary', 'js-post-summary') data-post-id='2003505' data-post-type-id='1'>, <Element 'div' id='question-summary-292357' class=('s-post-summary', 'js-post-summary') data-post-id='292357' data-post-type-id='1'>, <Element 'div' id='question-summary-231767' class=('s-post-summary', 'js-post-summary') data-post-id='231767' data-post-type-id='1'>, <Element 'div' id='question-summary-5767325' class=('s-post-summary', 'js-post-summary') data-post-id='5767325' data-post-type-id='1'>, <Element 'div' id='question-summary-477816' class=('s-post-summary', 'js-post-summary') data-post-id='477816' data-post-type-id='1'>, <Element 'div' id='question-summary-6591213' class=('

In [7]:
print(question_summaries[1].text)

25719 votes
104 answers
12.9m views
How do I undo the most recent local commits in Git?
I accidentally committed the wrong files to Git, but didn't push the commit to the server yet. How do I undo those commits from the local repository?
git
version-control
repository
git-commit
undo
Community wiki
93 revs, 64 users 11%
Peter Mortensen


In [8]:
"""columns = ['votes', 'answers', 'views', 'question', 'description', 'tags', 'user', 'user_details', 'date']"""

"columns = ['votes', 'answers', 'views', 'question', 'description', 'tags', 'user', 'user_details', 'date']"

In [9]:
this_row =list(question_summaries[1].text.split('\n'))
this_row

['25719 votes',
 '104 answers',
 '12.9m views',
 'How do I undo the most recent local commits in Git?',
 "I accidentally committed the wrong files to Git, but didn't push the commit to the server yet. How do I undo those commits from the local repository?",
 'git',
 'version-control',
 'repository',
 'git-commit',
 'undo',
 'Community wiki',
 '93 revs, 64 users 11%',
 'Peter Mortensen']

In [10]:
#len(this_row)==len(columns)
key_names = ['user', 'question','votes','date']
key_names = ['tags']
classes_needed = ['.s-user-card--link','.s-link','.s-post-summary--stats-item-number','.relativetime']
classes_needed = ['.s-post-summary--meta-tags']

In [11]:

this_question_element = question_summaries[0]
this_question_element.find('.s-post-summary--meta-tags', first=True).text.replace('\n',' and ')
#import numpy as np
#import pandas as pd

In [12]:
this_question_element.find('.s-post-summary--meta-tags', first=True).text.replace('\n',' and ')


'java and c++ and performance and cpu-architecture and branch-prediction'

In [13]:
"""
data = []

for items in question_summaries:
    ques_data = {}
    for i, elements in enumerate(classes_needed):
        sub_elements = items.find(elements, first = True)
        keyname = key_names[i]
        if sub_elements:
            ques_data[keyname]=sub_elements.text.replace('\n',' and ')
        else:
            ques_data[keyname]=np.nan
    data.append(ques_data)

data[0]"""

"\ndata = []\n\nfor items in question_summaries:\n    ques_data = {}\n    for i, elements in enumerate(classes_needed):\n        sub_elements = items.find(elements, first = True)\n        keyname = key_names[i]\n        if sub_elements:\n            ques_data[keyname]=sub_elements.text.replace('\n',' and ')\n        else:\n            ques_data[keyname]=np.nan\n    data.append(ques_data)\n\ndata[0]"

In [14]:
def clean_data(text, keyname=None):
    if keyname == 'tags':
        return text.replace('\n', ' and ')
    return text

In [15]:
def parse_tagged_page(html):
    question_summaries = html.find(".s-post-summary") 
    key_names = ['user', 'votes','tags','date']
    class_needed = ['.s-user-card--link','.s-post-summary--stats-item-number','.s-post-summary--meta-tags','.relativetime']
    data = []

    for items in question_summaries:
        q_data = {}
        for i, elements in enumerate(class_needed):
            sub_elements = items.find(elements, first = True)
            keyname = key_names[i]
            if sub_elements:
                q_data[keyname]=clean_data(sub_elements.text.replace('\n',' and '), keyname=keyname)
            else:
                q_data[keyname]=clean_data(np.nan, keyname=keyname)
        data.append(q_data)
    return data

In [16]:
def extract_data_from_url(url):
    r = requests.get(url)
    if r.status_code not in range(200, 299):
        return []
    html_str = r.text
    html = HTML(html=html_str)
    data = parse_tagged_page(html)
    return data

In [17]:
def scrape_tag(query_filter = "Votes", max_pages=1000, pagesize=50):
    base_url = 'https://stackoverflow.com/questions'
    data = []
    for p in range(max_pages):
        page_num = p + 1
        url = f"{base_url}?tab={query_filter}&page={page_num}&pagesize={pagesize}"
        data += extract_data_from_url(url)
        time.sleep(1.3)
    return data

In [18]:
data = scrape_tag(query_filter='Votes', max_pages=1000, pagesize=50)#Unanswered

In [19]:
data[:10]

[{'user': 'GManNickG',
  'votes': '27015',
  'tags': 'java and c++ and performance and cpu-architecture and branch-prediction',
  'date': 'Jun 27, 2012 at 13:51'},
 {'user': 'Community wiki',
  'votes': '25719',
  'tags': 'git and version-control and repository and git-commit and undo',
  'date': nan},
 {'user': 'Matthew Rankin',
  'votes': '20176',
  'tags': 'git and version-control and git-branch and git-push and git-remote',
  'date': 'Jan 5, 2010 at 1:12'},
 {'user': 'pupeno',
  'votes': '13636',
  'tags': 'git and version-control and git-pull and git-fetch',
  'date': 'Nov 15, 2008 at 9:51'},
 {'user': 'Alex. S.',
  'votes': '12644',
  'tags': 'python and iterator and generator and yield',
  'date': 'Oct 23, 2008 at 22:21'},
 {'user': 'Walker',
  'votes': '11528',
  'tags': 'javascript and arrays',
  'date': 'Apr 23, 2011 at 22:17'},
 {'user': 'Oli',
  'votes': '11431',
  'tags': 'json and mime-types and content-type',
  'date': 'Jan 25, 2009 at 15:25'},
 {'user': 'Forrest',
  'vo

In [20]:
df= pd.DataFrame(data)

In [21]:
df.to_csv('E:\stg_ques_votes.csv', index=False, header=True, sep=',')

In [22]:
df_csv = pd.read_csv('E:\stg_ques_votes.csv',sep=',')

In [23]:
df_csv.head()

,user,votes,tags,date
0,GManNickG,27014,java and c++ and performance and cpu-architect...,"Jun 27, 2012 at 13:51"
1,Community wiki,25715,git and version-control and repository and git...,NaN
2,Matthew Rankin,20167,git and version-control and git-branch and git...,"Jan 5, 2010 at 1:12"
3,pupeno,13632,git and version-control and git-pull and git-f...,"Nov 15, 2008 at 9:51"
4,Alex. S.,12640,python and iterator and generator and yield,"Oct 23, 2008 at 22:21"


In [24]:
data = scrape_tag(query_filter='Unanswered',max_pages=1000, pagesize=50)#Unanswered

In [25]:
data[:10]

[{'user': 'Joey.Z',
  'votes': '89',
  'tags': 'android and android-ndk and gdb and gdbserver',
  'date': 'Feb 5, 2018 at 3:03'},
 {'user': 'Philip',
  'votes': '88',
  'tags': 'julia and traits',
  'date': 'Feb 24, 2016 at 22:19'},
 {'user': 'dad',
  'votes': '79',
  'tags': 'r and ggplot2 and sf and xquartz',
  'date': 'Jul 1, 2018 at 18:42'},
 {'user': 'dMurdZ',
  'votes': '65',
  'tags': 'xcode and interface-builder and xcode6',
  'date': 'Sep 16, 2014 at 13:32'},
 {'user': 'Murtaza Hussain',
  'votes': '62',
  'tags': 'reactjs and facebook and webview and stripe-payments and instagram',
  'date': 'Jun 3, 2022 at 10:30'},
 {'user': 'Mohammad ZeinEddin',
  'votes': '62',
  'tags': 'php and doctrine-orm and zend-framework2 and zend-form2',
  'date': 'Jul 18, 2013 at 11:39'},
 {'user': 'UserFrank',
  'votes': '52',
  'tags': 'amazon-s3 and upload and cpu and paperclip and space-leak',
  'date': 'Aug 15, 2013 at 1:14'},
 {'user': 'Richie',
  'votes': '51',
  'tags': 'android and google

In [26]:
df= pd.DataFrame(data)
df.to_csv('E:\stg_ques_unans.csv', index=False, header=True, sep=',')

In [28]:
df.shape

(37500, 4)

In [29]:
data = scrape_tag(query_filter='Newest',max_pages=1000, pagesize=50)#Unanswered

In [30]:
df= pd.DataFrame(data)
df.to_csv('E:\stg_ques_new.csv', index=False, header=True, sep=',')